In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("../dataset/dataset(no tf-idf).csv")

In [3]:
# DATE TO TIME (!to delete!)
time = []
for index, row in df.iterrows():
    time.append(datetime.strptime(row.date, "%Y-%m-%d %H:%M:%S").hour)

df = df.replace(df.date.to_list(), time)
df = df.rename(columns={'date': 'time'})

In [4]:
X = df[['time', 'len_plain_text', 'question_marks', 'esclamation_marks', 'emojis', 'hashtags', 'tags', 'urls']]
y = df[['ratio']]

RANDOM FOREST

In [5]:
# TRAINING

rf_clf = RandomForestRegressor(n_estimators=100, oob_score=True)

kf = KFold(n_splits=10)
rf_mses = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    rf_clf.fit(X_train, y_train.values.ravel())
    rf_predictions = rf_clf.predict(X_test)
    rf_mses.append(mean_squared_error(y_test.values.ravel(), rf_predictions))

In [6]:
# VARIABLE IMPORTANCE

for i in range(0,len(X.columns)):
    print(X.columns[i], ": ", rf_clf.feature_importances_[i])

time :  0.06365182035010605
len_plain_text :  0.11180698196361921
question_marks :  0.005729841816005986
esclamation_marks :  0.023824799159756438
emojis :  0.425066154504113
hashtags :  0.10201235996998102
tags :  0.009474045583373902
urls :  0.25843399665304445


In [7]:
# RF MSE mean and variance

rf_mean = 0
for mse in rf_mses:
    rf_mean += mse
rf_mean = rf_mean/len(rf_mses)

rf_variance = 0
for mse in rf_mses:
    rf_variance += ((mse - rf_mean)**2)
rf_variance = rf_variance/len(rf_mses)
    
print("MSE mean: ", rf_mean)
print("MSE variance: ", rf_variance)

MSE mean:  1.663191653363822e-05
MSE variance:  8.421386220829348e-10


In [8]:
# OOB Score

print(rf_clf.oob_score_)

0.6128834340159781


SVR

In [9]:
# DATASET STANDARDIZATION

df = df[['time', 'len_plain_text', 'question_marks', 'esclamation_marks', 'emojis', 'hashtags', 'tags', 'urls', 'ratio']]
scaler = StandardScaler()
df = scaler.fit_transform(df)

In [19]:
X = df.loc[,0:7]
y = df[8]

SyntaxError: invalid syntax (<ipython-input-19-cd6226adab6a>, line 1)

In [15]:
df

array([[ 0.8968618 , -0.32086756,  2.01355853, ..., -0.22335005,
         1.41759498,  2.70345867],
       [-0.41062349,  0.33951648,  2.01355853, ..., -0.22335005,
         1.41759498,  2.15612928],
       [-1.15775794,  1.08795173, -0.43288229, ..., -0.22335005,
         1.41759498,  2.80608293],
       ...,
       [ 0.71007818,  1.17600293, -0.43288229, ..., -0.22335005,
         1.41759498,  0.42068252],
       [ 0.8968618 ,  1.63827176, -0.43288229, ..., -0.22335005,
         1.41759498, -0.24877357],
       [ 0.71007818,  0.99990052, -0.43288229, ..., -0.22335005,
         1.41759498, -0.30456158]])

In [ ]:
# TRAINING

svr_clf = LinearSVR()
kf = KFold(n_splits=10)

svr_mses = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    svr_clf.fit(X_train, y_train.values.ravel())
    svr_predictions = svr_clf.predict(X_test)
    svr_mses.append(mean_squared_error(y_test.values.ravel(), svr_predictions))
    

In [ ]:
# SVR MSE mean and variance

svr_mean = 0
for mse in svr_mses:
    svr_mean += mse
svr_mean = svr_mean/len(svr_mses)

svr_variance = 0
for mse in svr_mses:
    svr_variance += ((mse - svr_mean)**2)
svr_variance = svr_variance/len(svr_mses)
    
print("MSE mean: ", svr_mean)
print("MSE variance: ", svr_variance)